In [1]:
import pandas as pd
df = pd.read_csv("Get_Personal_Export.csv")
df

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,...,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment References
0,#11172,truffy88@hotmail.com,paid,2024-11-01 22:52:44 +0000,unfulfilled,NaN,yes,GBP,7.20,1.99,...,NaN,NaN,NaN,NaN,NaN,NaN,Wales,England,rNjHm0nKzKCsc7fMkjNpkd3Im,rNjHm0nKzKCsc7fMkjNpkd3Im
1,#11172,truffy88@hotmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,#11172,truffy88@hotmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,#11172,truffy88@hotmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,#11172,truffy88@hotmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11435,#6678,emmaclarke23112@gmail.com,paid,2023-11-01 00:43:28 +0000,fulfilled,2023-11-02 16:42:53 +0000,yes,GBP,37.39,3.49,...,NaN,NaN,NaN,NaN,NaN,NaN,Wales,Wales,rSniQgjuc8vb2r6RYcSqLKs6y + ruKhe2pYzdGrKWnOaQ...,rSniQgjuc8vb2r6RYcSqLKs6y + ruKhe2pYzdGrKWnOaQ...
11436,#6678,emmaclarke23112@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11437,#6678,emmaclarke23112@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11438,#6677,sgonsalves2013@googlemail.com,paid,2023-11-01 00:08:31 +0000,fulfilled,2023-11-02 16:41:17 +0000,yes,GBP,31.00,3.49,...,NaN,NaN,NaN,NaN,NaN,NaN,England,England,rOtJN5o4pw2nQ1cyMvUnBDpVi + rO0SBmSILLMxeosUQZ...,rOtJN5o4pw2nQ1cyMvUnBDpVi + rO0SBmSILLMxeosUQZ...


In [3]:
print(df.columns)

Index(['Name', 'Email', 'Financial Status', 'Paid at', 'Fulfillment Status',
       'Fulfilled at', 'Accepts Marketing', 'Currency', 'Subtotal', 'Shipping',
       'Taxes', 'Total', 'Discount Code', 'Discount Amount', 'Shipping Method',
       'Created at', 'Lineitem quantity', 'Lineitem name', 'Lineitem price',
       'Lineitem compare at price', 'Lineitem sku',
       'Lineitem requires shipping', 'Lineitem taxable',
       'Lineitem fulfillment status', 'Billing Name', 'Billing Street',
       'Billing Address1', 'Billing Address2', 'Billing Company',
       'Billing City', 'Billing Zip', 'Billing Province', 'Billing Country',
       'Billing Phone', 'Shipping Name', 'Shipping Street',
       'Shipping Address1', 'Shipping Address2', 'Shipping Company',
       'Shipping City', 'Shipping Zip', 'Shipping Province',
       'Shipping Country', 'Shipping Phone', 'Notes', 'Note Attributes',
       'Cancelled at', 'Payment Method', 'Payment Reference',
       'Refunded Amount', 'Vendor', '

In [5]:
df = df.rename(columns={
    'Lineitem quantity': 'Product quantity',
    'Lineitem name': 'Product name',
    'Name': 'OrderID',
    'Created at': 'OrderDate',
    'Lineitem price': 'Product price'
})

In [7]:
df['OrderDate'] = pd.to_datetime(df['OrderDate'], errors='coerce', utc=True).dt.date

In [9]:
#select the order id for rwos where the cancelled column is not empty
cancelled_order_ids = df.loc[df['Cancelled at'].notnull(), 'OrderID'].unique()

# filter dataframe to delete rows with these OrderIDs
df_cleaned = df[~df['OrderID'].isin(cancelled_order_ids)]

In [11]:
#difined the pharses i want to remove from product names to standardise them
phrases_to_remove = [
    r'PRE ORDER FOR 30TH SEPT -',  
    r'PRE-ORDER FOR 30th SEPT',     
    r'Launching \d{2}/\d{2}/\d{2}', 
    r'LAUNCHES \d{1,2}\.\d{1,2}\.\d{2,4}', #2/2/24"
    r'BRAND NEW\*',               
    r'PRE ORDER'                 
]

#This creates a regex pattern   r'(?i)'   is case insensitive flag this the matches values even if its in upper case or lower case
pattern = r'(?i)' + '|'.join(phrases_to_remove)

# Remove the unwanted phrases from Product name column
df['Product name'] = df['Product name'].str.replace(pattern, '', regex=True).str.strip()

#save
df.to_csv("Normalised_Dataset.csv", index=False)

In [13]:
columns_to_remove = ['Financial Status', 'Paid at', 'Fulfillment Status', 'Fulfilled at', 'Accepts Marketing','Taxes', 'Shipping Method','Lineitem compare at price', 'Lineitem sku','Lineitem requires shipping', 'Lineitem taxable', 'Lineitem fulfillment status', 'Billing Company',
       'Billing Phone', 'Shipping Company', 'Shipping Phone', 'Notes', 'Note Attributes', 'Refunded Amount', 'Vendor' , 'Id', 'Tags', 'Risk Level', 'Source',
       'Lineitem discount', 'Tax 1 Name', 'Tax 1 Value', 'Tax 2 Name',
       'Tax 2 Value', 'Tax 3 Name', 'Tax 3 Value', 'Tax 4 Name', 'Tax 4 Value',
       'Tax 5 Name', 'Tax 5 Value', 'Phone', 'Receipt Number', 'Duties',
       'Billing Province Name', 'Shipping Province Name', 'Payment References', 'Currency', 'Discount Code', 'Payment Reference', 'Billing Country',
       'Shipping Country', 'Payment ID']
df = df.drop(columns_to_remove, axis=1)

In [15]:
columns_to_drop = [
    'Shipping Name',
    'Shipping Street',
    'Shipping Address1',
    'Shipping Address2',
    'Shipping City',
    'Shipping Zip',
    'Shipping Province'
]

df = df.drop(columns=columns_to_drop)

# save
df.to_csv("BI_629703.csv", index=False)

In [17]:
import pandas as pd
df = pd.read_csv("BI_629703.csv")
df

,OrderID,Email,Subtotal,Shipping,Total,Discount Amount,OrderDate,Product quantity,Product name,Product price,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing City,Billing Zip,Billing Province,Cancelled at,Payment Method
0,#11172,truffy88@hotmail.com,7.20,1.99,9.19,1.71,2024-11-01,2,Glass Breaker & seatbelt cutter Rescue Tool ke...,0.99,Jessica Thomas-Thompson,"26, Alexandra Road",26,Alexandra Road,Ynysddu,NP11 7JZ,WLS,NaN,Shopify Payments
1,#11172,truffy88@hotmail.com,NaN,NaN,NaN,NaN,2024-11-01,2,Glass Breaker & seatbelt cutter Rescue Tool ke...,0.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,#11172,truffy88@hotmail.com,NaN,NaN,NaN,NaN,2024-11-01,2,Glass Breaker & seatbelt cutter Rescue Tool ke...,0.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,#11172,truffy88@hotmail.com,NaN,NaN,NaN,NaN,2024-11-01,2,Glass Breaker & seatbelt cutter Rescue Tool ke...,0.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,#11172,truffy88@hotmail.com,NaN,NaN,NaN,NaN,2024-11-01,1,Glass Breaker & seatbelt cutter Rescue Tool ke...,0.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11435,#6678,emmaclarke23112@gmail.com,37.39,3.49,44.18,6.59,2023-11-01,1,Multi Moo Uk Legal Safety keychain,21.99,emma clarke,"78 Gelligaled Road, Ystrad",78 Gelligaled Road,Ystrad,Pentre,CF41 7RQ,WLS,NaN,Shopify Payments
11436,#6678,emmaclarke23112@gmail.com,NaN,NaN,NaN,NaN,2023-11-01,1,Tip,3.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11437,#6678,emmaclarke23112@gmail.com,NaN,NaN,NaN,NaN,2023-11-01,1,Mi Amor UK Legal Safety Keychain Set,21.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11438,#6677,sgonsalves2013@googlemail.com,31.00,3.49,34.49,8.98,2023-11-01,1,Lemonella Safety Keychain,19.99,Sharon Gonsalves,"29 Bridge Street, Long Eaton","29 Bridge Street, Long Eaton",NaN,Nottinghamshire,NG10 4QS,ENG,NaN,Shopify Payments


In [29]:
columns_to_fill = [
    'Subtotal', 'Shipping', 'Total', 'Discount Amount',
    'Billing Name', 'Billing Street', 
    'Billing Address1', 'Billing Address2', 'Billing City',
    'Billing Zip', 'Billing Province', 'Payment Method'
]

pd.set_option('future.no_silent_downcasting', True)

# FILL COLumns based on orderid
df[columns_to_fill] = (
    df.groupby('OrderID', group_keys=False)[columns_to_fill]
    .apply(lambda group: group.ffill().bfill())  # Forward-fill and backward-fill
    .infer_objects(copy=False)  # Ensure types remain consistent
)

# Save
df.to_csv('BI_629703.csv', index=False)

In [31]:
df.drop('Cancelled at', axis=1, inplace=True)

In [33]:
# Save
df.to_csv('BI_629703.csv', index=False)

In [38]:
#create customers table
customers = df[['Email', 'Billing Name', 'Billing Street', 'Billing Address1', 
                  'Billing Address2', 'Billing City', 'Billing Zip', 'Billing Province']].drop_duplicates().reset_index(drop=True)
customers['CustomerID'] = customers.index + 1
customers = customers[['CustomerID', 'Email', 'Billing Name', 'Billing Street', 
                       'Billing Address1', 'Billing Address2', 'Billing City', 
                       'Billing Zip', 'Billing Province']]
customers.to_csv("customers.csv", index=False)

In [40]:
#locations table
locations = df[['Billing City']].drop_duplicates().reset_index(drop=True)
locations['LocationID'] = locations.index + 1
locations = locations[['LocationID', 'Billing City']]
locations.to_csv("locations.csv", index=False)

In [42]:
#payments table
payments = df[['Payment Method']].drop_duplicates().reset_index(drop=True)
payments['PaymentID'] = payments.index + 1
payments = payments[['PaymentID', 'Payment Method']]
payments.to_csv("paymentmethod.csv", index=False)

In [44]:
#dATE table
date = df[['OrderDate']].drop_duplicates().reset_index(drop=True)
date['DateID'] = date.index + 1
date = date[['DateID', 'OrderDate']]
date.to_csv("date.csv", index=False)

In [46]:
#create products table
products = df[['Product name', 'Product price']].drop_duplicates().reset_index(drop=True)
products['ProductID'] = products.index + 1
products = products[['ProductID', 'Product name', 'Product price']]

products.to_csv("products.csv", index=False)

In [48]:
import pandas as pd
df = pd.read_csv("BI_629703.csv")
df

,OrderID,Email,Subtotal,Shipping,Total,Discount Amount,OrderDate,Product quantity,Product name,Product price,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing City,Billing Zip,Billing Province,Payment Method
0,#11172,truffy88@hotmail.com,7.20,1.99,9.19,1.71,2024-11-01,2,Glass Breaker & seatbelt cutter Rescue Tool ke...,0.99,Jessica Thomas-Thompson,"26, Alexandra Road",26,Alexandra Road,Ynysddu,NP11 7JZ,WLS,Shopify Payments
1,#11172,truffy88@hotmail.com,7.20,1.99,9.19,1.71,2024-11-01,2,Glass Breaker & seatbelt cutter Rescue Tool ke...,0.99,Jessica Thomas-Thompson,"26, Alexandra Road",26,Alexandra Road,Ynysddu,NP11 7JZ,WLS,Shopify Payments
2,#11172,truffy88@hotmail.com,7.20,1.99,9.19,1.71,2024-11-01,2,Glass Breaker & seatbelt cutter Rescue Tool ke...,0.99,Jessica Thomas-Thompson,"26, Alexandra Road",26,Alexandra Road,Ynysddu,NP11 7JZ,WLS,Shopify Payments
3,#11172,truffy88@hotmail.com,7.20,1.99,9.19,1.71,2024-11-01,2,Glass Breaker & seatbelt cutter Rescue Tool ke...,0.99,Jessica Thomas-Thompson,"26, Alexandra Road",26,Alexandra Road,Ynysddu,NP11 7JZ,WLS,Shopify Payments
4,#11172,truffy88@hotmail.com,7.20,1.99,9.19,1.71,2024-11-01,1,Glass Breaker & seatbelt cutter Rescue Tool ke...,0.99,Jessica Thomas-Thompson,"26, Alexandra Road",26,Alexandra Road,Ynysddu,NP11 7JZ,WLS,Shopify Payments
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11435,#6678,emmaclarke23112@gmail.com,37.39,3.49,44.18,6.59,2023-11-01,1,Multi Moo Uk Legal Safety keychain,21.99,emma clarke,"78 Gelligaled Road, Ystrad",78 Gelligaled Road,Ystrad,Pentre,CF41 7RQ,WLS,Shopify Payments
11436,#6678,emmaclarke23112@gmail.com,37.39,3.49,44.18,6.59,2023-11-01,1,Tip,3.30,emma clarke,"78 Gelligaled Road, Ystrad",78 Gelligaled Road,Ystrad,Pentre,CF41 7RQ,WLS,Shopify Payments
11437,#6678,emmaclarke23112@gmail.com,37.39,3.49,44.18,6.59,2023-11-01,1,Mi Amor UK Legal Safety Keychain Set,21.99,emma clarke,"78 Gelligaled Road, Ystrad",78 Gelligaled Road,Ystrad,Pentre,CF41 7RQ,WLS,Shopify Payments
11438,#6677,sgonsalves2013@googlemail.com,31.00,3.49,34.49,8.98,2023-11-01,1,Lemonella Safety Keychain,19.99,Sharon Gonsalves,"29 Bridge Street, Long Eaton","29 Bridge Street, Long Eaton",NaN,Nottinghamshire,NG10 4QS,ENG,Shopify Payments


In [86]:
#fact1
factsales11 = df.merge(customers, on=['Email', 'Billing Name', 'Billing Street', 'Billing Address1', 
                                 'Billing Address2', 'Billing City', 'Billing Zip', 'Billing Province']) \
            .merge(products, on=['Product name', 'Product price']) \
            .merge(locations, on=['Billing City']) \
            .merge(payments, on=['Payment Method']) \
            .merge(date, on=['OrderDate'])

factsales11 = factsales11[['OrderID', 'Subtotal', 'Total', 'Discount Amount', 
                    'CustomerID', 'ProductID', 'LocationID', 'PaymentID', 'DateID', 'Product quantity']]
factsales11.to_csv("factsales11.csv", index=False)

In [80]:
factsales1 = df.merge(customers, on=['Email', 'Billing Name', 'Billing Street', 'Billing Address1', 
                                     'Billing Address2', 'Billing City', 'Billing Zip', 'Billing Province']) \
    .merge(products, on=['Product name', 'Product price']) \
    .merge(locations, on=['Billing City']) \
    .merge(payments, on=['Payment Method']) \
    .merge(date, on=['OrderDate'])

factsales1_grouped = factsales1.groupby(['OrderID', 'Subtotal', 'Total', 'Discount Amount', 
                                        'CustomerID', 'LocationID', 'PaymentID', 'DateID'], as_index=False) \
    .agg({
        'ProductID': lambda x: list(x),  
        'Product quantity': lambda x: list(x)  ##merge the product id and product quantity to avoid duplicates 
    })
factsales1_grouped.to_csv("factsales1.csv", index=False)

In [82]:
factsales1_grouped = factsales1.groupby(['OrderID', 'Subtotal', 'Total', 'Discount Amount', 
                                        'CustomerID', 'LocationID', 'PaymentID', 'DateID'], as_index=False) \
    .agg({
        'ProductID': lambda x: ', '.join(map(str, x)),  #COMMA sepearex string 
        'Product quantity': lambda x: ', '.join(map(str, x))  # merge product quantites 
    })

# Save the grouped and cleaned data to CSV
factsales1_grouped.to_csv("factsales1.csv", index=False)

In [90]:
factsales1 = df.merge(customers, on=['Email', 'Billing Name', 'Billing Street', 'Billing Address1', 
                                     'Billing Address2', 'Billing City', 'Billing Zip', 'Billing Province']) \
    .merge(products, on=['Product name', 'Product price']) \
    .merge(locations, on=['Billing City']) \
    .merge(payments, on=['Payment Method']) \
    .merge(date, on=['OrderDate'])
factsales1['FactID'] = range(1, len(factsales1) + 1)
factsales1 = factsales1[['FactID', 'OrderID', 'Subtotal', 'Total', 'Discount Amount', 
                         'CustomerID', 'ProductID', 'LocationID', 'PaymentID', 'DateID', 'Product quantity']]
factsales1.to_csv("factsales1.csv", index=False)